In [19]:
import os
from os.path import exists
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
# from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

In [17]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

## Part 1: Model Architecture

### Model Architecture

The Encoder maps an input sequence of symbol representations $(x_1, \ldots, x_n)$ to a sequence continuous representations $\textbf{z}=(z_1,\ldots, z_n)$. Given $\textbf{z}$.

The decoder then generates an output sequence $(y_1,\ldots,y_n)$ of symbols one element at a time. At each step the model is *auto-regressive*, _consuming the previously generated symbols as additional input_ when generating the next.

In [18]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        ' Take in and process masked scr and target sequences. '
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_embed):
        return self.encoder(self.src_embed(src), src_embed)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

[`F.log_softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.log_softmax.html)

Apply a softmax followed by a logarithm.

While mathematically equivalent to `log(softmax(x))`, __doing these two operations separately is slower and numerically unstable__. This function uses an alternative formulation to _compute the output and gradient correctly_.

$$
\text{LogSoftmax}(x_i) = \log \frac{\exp(x_i)}{\sum_j\exp(x_j)}
$$

In [20]:
class Generator(nn.Module):
    " Define standard Linear + Softmax generation step "

    def __init__(self, d_model, vocab):
        super().__init__()
        # project from d_model to vocab_size
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        # log_softmax on last dim (vocab_size) to get prob_distrib
        return F.log_softmax(self.proj(x), dim=-1)  # (batch_size, sequence_length, vocab_size)

### Encoder and Decoder Stacks

#### Encoder

The encoder is composed of a stack of $N=6$ identical layers.

In [21]:
def clones(module, N):
    ' clone N same module '
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

#### Layer Normalization

Reduce the covariate shift problem from Batch Norm by _fixing the mean and the variance of the summed inputs within each layer_.

Thus, compute the layer normalization statistics over all the elements in the same layer:

$$
\begin{align}
LN(\textbf{z};\alpha, \beta) &= \frac{(\textbf{z} - \mu)}{\sigma}\odot\alpha + \beta \\
\mu = \frac{1}{D}\sum_{i=1}^Dz_i, &\ \sigma = \sqrt{\frac{1}{D}\sum_{i=1}(z_i - \mu)^2}
\end{align}
$$

where $z_i$ is the $i$-th element of the vector $\textbf{z}$.


In [PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html),

`torch.nn.LayerNorm(normalized_shape, eps=1e-05, elementwise_affine=True, bias=True, device=None, dtype=None)`

which implement the same equation:

$$
y = \frac{x - \textbf{E}[x]}{\sqrt{\textbf{Var}[x] + \epsilon}} * \gamma + \beta
$$

The mean and standard-deviation are calculated over _the last $D$ dimensions_, where $D$ is the dimension of `normalized_shape`.

$\gamma$ and $\beta$ are _learnable affine transform parameters_ of normalized_shape if `elementwise_affine` is True. 

In [23]:
class LayerNorm(nn.Module):
    ' Construct a Layer Norm module '
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [22]:
class Encoder(nn.Module):
    ' Core encoder '
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        # self.ln = nn.LayerNorm(layer.size)
        self.ln = LayerNorm(layer.size)

    def forward(self, x, mask):
        ' Pass the input (and mask) through each layer in turn '
        for layer in self.layers:
            x = layer(x, mask)
        return self.ln(x)

In [24]:
class SublayerConnection(nn.Module):
    '''
    A residual connection followed by a layer norm.
    '''

    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = dropout

    def forward(self, x, sublayer):
        " Apply the residual connection to any sublayer with the same size. "
        ' Apply dropout to the output of each sublayer before added to the usublayer input and normalized '
        return x + self.dropout(sublayer(self.norm(x)))

In [28]:
class EncoderLayer(nn.Module):
    '''
    Each encoder layer has 2 sublayers: a multi-head attention and a feed forward
    '''
    def __init__(self, size, self_atten, ffn, dropout):
        super().__init__()
        self.self_atten = self_atten
        self.ffn = ffn
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
    
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_atten(x, x, x, mask))
        return self.sublayer[1](x, self.ffn)

#### Decoder

The decoder is also composed of a stack of $N=6$ identical layers.

In [27]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


In [29]:
class DecoderLayer(nn.Module):
    ' Decoder is made of self-atten (masked), scr_atten, and FFN '
    def __init__(self, size, self_atten, scr_atten, ffn, dropout):
        super().__init__()
        self.size = size
        self.self_atten = self_atten
        self.src_atten = scr_atten
        self.ffn = ffn
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, scr_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_atten(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_atten(x, m, m, scr_mask))
        return self.sublayer[2](x, self.ffn)